In [2]:
import os

import pandas as pd
import numpy as np
import requests
import json
import csv 
from pprint import pprint

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

import sqlite3 as sq

# Create Math Dataframe and save as CSV 

In [3]:
# Convert Math csv to DataFrame 
math_csv = "dataMathNational.csv"
math_data_df = pd.read_csv(math_csv)
math_data_df.head()

,id,status,result__focalYear,result__focalSample,result__targetYear,result__targetSample,result__statType,result__subject,result__grade,result__jurisdiction,...,result__valValue,result__focalValue,result__isFocalStatDisplayable,result__focalErrorFlag,result__targetValue,result__isTargetStatDisplayable,result__targetErrorFlag,result__isSigDisplayable,result__gap,result__sig
0,1,200.0,2009,R3,2017,R3,MN:MN,MAT,4,AK,...,1,238.445992,1,0,230.451760,1,0,1,7.994232,HIGHER
1,2,NaN,2009,R3,2017,R3,MN:MN,MAT,4,AK,...,2,235.929342,1,0,230.461187,1,0,1,5.468155,HIGHER
2,3,NaN,2009,R3,2017,R3,MN:MN,MAT,4,AL,...,1,227.718193,1,0,232.458570,1,0,1,-4.740377,LOWER
3,4,NaN,2009,R3,2017,R3,MN:MN,MAT,4,AL,...,2,228.224749,1,0,231.878482,1,0,1,-3.653733,LOWER
4,5,NaN,2009,R3,2017,R3,MN:MN,MAT,4,AR,...,1,238.824583,1,0,234.980702,1,0,1,3.843880,HIGHER


In [4]:
# Clean Math DataFrame 
math_data_df = math_data_df[['id','result__jurisdiction', 'result__valLabel',  'result__focalValue', 'result__targetValue', 'result__gap']]
math_data_df.head()

,id,result__jurisdiction,result__valLabel,result__focalValue,result__targetValue,result__gap
0,1,AK,Male,238.445992,230.451760,7.994232
1,2,AK,Female,235.929342,230.461187,5.468155
2,3,AL,Male,227.718193,232.458570,-4.740377
3,4,AL,Female,228.224749,231.878482,-3.653733
4,5,AR,Male,238.824583,234.980702,3.843880


In [5]:
# Rename Columns 
math_data_df.columns = ["id","state", "gender", "avg_2009_mathScores", "avg_2017_mathScores", "mathScores_precentChange"]

In [6]:
# View updated Dataframe
math_data_df.head()

,id,state,gender,avg_2009_mathScores,avg_2017_mathScores,mathScores_precentChange
0,1,AK,Male,238.445992,230.451760,7.994232
1,2,AK,Female,235.929342,230.461187,5.468155
2,3,AL,Male,227.718193,232.458570,-4.740377
3,4,AL,Female,228.224749,231.878482,-3.653733
4,5,AR,Male,238.824583,234.980702,3.843880


In [7]:
# Create Math DataFrame for Males only
male_math_data = math_data_df.loc[math_data_df['gender'] == "Male"]
male_math_data.head()

,id,state,gender,avg_2009_mathScores,avg_2017_mathScores,mathScores_precentChange
0,1,AK,Male,238.445992,230.451760,7.994232
2,3,AL,Male,227.718193,232.458570,-4.740377
4,5,AR,Male,238.824583,234.980702,3.843880
6,7,AZ,Male,230.283950,235.631656,-5.347706
8,9,CA,Male,232.613832,233.265686,-0.651854


In [8]:
# male_math_data.to_csv("male_math_data.csv", index=False)

In [9]:
# Create Math Datae for Females only 
female_math_data = math_data_df.loc[math_data_df['gender'] == "Female"]
female_math_data.head()

,id,state,gender,avg_2009_mathScores,avg_2017_mathScores,mathScores_precentChange
1,2,AK,Female,235.929342,230.461187,5.468155
3,4,AL,Female,228.224749,231.878482,-3.653733
5,6,AR,Female,236.199533,232.664988,3.534545
7,8,AZ,Female,229.686859,233.215046,-3.528187
9,10,CA,Female,230.676542,231.215506,-0.538964


In [10]:
# female_math_data.to_csv("female_math_data.csv", index=False)

# Create Reading Dataframe and save as CSV

In [11]:
# Convert Reading csv to DataFrame 
reading_csv = "dataReadNational.csv"
reading_data_df = pd.read_csv(reading_csv)
reading_data_df.head()

,id,status,result__focalYear,result__focalSample,result__targetYear,result__targetSample,result__statType,result__subject,result__grade,result__jurisdiction,...,result__valValue,result__focalValue,result__isFocalStatDisplayable,result__focalErrorFlag,result__targetValue,result__isTargetStatDisplayable,result__targetErrorFlag,result__isSigDisplayable,result__gap,result__sig
0,1,200.0,2009,R3,2017,R3,MN:MN,RED,4,AK,...,1,206.724229,1,0,203.051302,1,0,1,3.672928,EQUAL
1,2,NaN,2009,R3,2017,R3,MN:MN,RED,4,AK,...,2,215.711234,1,0,211.343343,1,0,1,4.367891,EQUAL
2,3,NaN,2009,R3,2017,R3,MN:MN,RED,4,AL,...,1,211.741931,1,0,212.821109,1,0,1,-1.079179,EQUAL
3,4,NaN,2009,R3,2017,R3,MN:MN,RED,4,AL,...,2,221.100699,1,0,220.006600,1,0,1,1.094099,EQUAL
4,5,NaN,2009,R3,2017,R3,MN:MN,RED,4,AR,...,1,210.975858,1,0,211.742620,1,0,1,-0.766762,EQUAL


In [12]:
# Clean Reading DataFrame 
reading_data_df = reading_data_df[['id','result__jurisdiction', 'result__valLabel',  'result__focalValue', 'result__targetValue', 'result__gap']]
reading_data_df.head()

,id,result__jurisdiction,result__valLabel,result__focalValue,result__targetValue,result__gap
0,1,AK,Male,206.724229,203.051302,3.672928
1,2,AK,Female,215.711234,211.343343,4.367891
2,3,AL,Male,211.741931,212.821109,-1.079179
3,4,AL,Female,221.100699,220.006600,1.094099
4,5,AR,Male,210.975858,211.742620,-0.766762


In [13]:
# Rename Columns 
reading_data_df.columns = ["id", "state", "gender", "avg_2009_readingScores", "avg_2017_readingScores", "readingScores_precentChange"]

In [14]:
# View updated Dataframe
reading_data_df.head()

,id,state,gender,avg_2009_readingScores,avg_2017_readingScores,readingScores_precentChange
0,1,AK,Male,206.724229,203.051302,3.672928
1,2,AK,Female,215.711234,211.343343,4.367891
2,3,AL,Male,211.741931,212.821109,-1.079179
3,4,AL,Female,221.100699,220.006600,1.094099
4,5,AR,Male,210.975858,211.742620,-0.766762


In [15]:
# Create Reading DataFrame for Males only
male_reading_data = reading_data_df.loc[reading_data_df['gender'] == "Male"]
male_reading_data.head()

,id,state,gender,avg_2009_readingScores,avg_2017_readingScores,readingScores_precentChange
0,1,AK,Male,206.724229,203.051302,3.672928
2,3,AL,Male,211.741931,212.821109,-1.079179
4,5,AR,Male,210.975858,211.742620,-0.766762
6,7,AZ,Male,206.913680,213.462670,-6.548990
8,9,CA,Male,207.055218,210.770608,-3.715390


In [16]:
# male_reading_data.to_csv("male_reading_data.csv", index=False)

In [17]:
# Create Reading DataFrame for Males only
female_reading_data = reading_data_df.loc[reading_data_df['gender'] == "Female"]
female_reading_data.head()

,id,state,gender,avg_2009_readingScores,avg_2017_readingScores,readingScores_precentChange
1,2,AK,Female,215.711234,211.343343,4.367891
3,4,AL,Female,221.100699,220.006600,1.094099
5,6,AR,Female,221.564513,220.633010,0.931503
7,8,AZ,Female,213.189225,217.520227,-4.331002
9,10,CA,Female,212.637646,220.293138,-7.655493


In [18]:
# female_reading_data.to_csv("female_reading_data.csv", index=False)

### Transform Math DataFrames

In [19]:
# Male Math Dataframe 
# Create a filtered dataframe from specific columns
male_math_cols = ["id","state", "gender", "avg_2009_mathScores", "avg_2017_mathScores", "mathScores_precentChange"]
male_math_transformed= male_math_data[male_math_cols].copy()

# Rename the column headers
male_math_transformed = male_math_transformed.rename(columns={"id": "id",
                                                          "state": "state",
                                                            "gender": "gender",
                                                            "avg_2009_mathScorese": "avg_2009_mathScorese",
                                                            "avg_2017_mathScores": "avg_2017_mathScores",
                                                          "mathScores_precentChange": "mathScores_precentChange"})

# Clean the data by dropping duplicates and setting the index
male_math_transformed.set_index("id", inplace=True)

male_math_transformed.head()

,state,gender,avg_2009_mathScores,avg_2017_mathScores,mathScores_precentChange
id,,,,,
1,AK,Male,238.445992,230.451760,7.994232
3,AL,Male,227.718193,232.458570,-4.740377
5,AR,Male,238.824583,234.980702,3.843880
7,AZ,Male,230.283950,235.631656,-5.347706
9,CA,Male,232.613832,233.265686,-0.651854


In [20]:
# Femal Math Dataframe 
# Create a filtered dataframe from specific columns
female_math_cols = ["id","state", "gender", "avg_2009_mathScores", "avg_2017_mathScores", "mathScores_precentChange"]
female_math_transformed= female_math_data[female_math_cols].copy()

# Rename the column headers
female_math_transformed = female_math_transformed.rename(columns={"id": "id",
                                                          "state": "state",
                                                            "gender": "gender",
                                                            "avg_2009_mathScorese": "avg_2009_mathScorese",
                                                            "avg_2017_mathScores": "avg_2017_mathScores",
                                                          "mathScores_precentChange": "mathScores_precentChange"})

# Clean the data by dropping duplicates and setting the index
female_math_transformed.set_index("id", inplace=True)

female_math_transformed.head()

,state,gender,avg_2009_mathScores,avg_2017_mathScores,mathScores_precentChange
id,,,,,
2,AK,Female,235.929342,230.461187,5.468155
4,AL,Female,228.224749,231.878482,-3.653733
6,AR,Female,236.199533,232.664988,3.534545
8,AZ,Female,229.686859,233.215046,-3.528187
10,CA,Female,230.676542,231.215506,-0.538964


### Transform Reading DataFrames

In [21]:
# Male Math Dataframe 
# Create a filtered dataframe from specific columns
male_reading_cols = ["id", "state", "gender", "avg_2009_readingScores", "avg_2017_readingScores", "readingScores_precentChange"]
male_reading_transformed= male_reading_data[male_reading_cols].copy()

# Rename the column headers
male_reading_transformed = male_reading_transformed.rename(columns={"id": "id",
                                                          "state": "state",
                                                            "gender": "gender",
                                                            "avg_2009_readingScores": "avg_2009_readingScores",
                                                            "avg_2017_readingScores": "avg_2017_readingScores",
                                                          "readingScores_precentChange": "readingScores_precentChange"})

# Clean the data by dropping duplicates and setting the index
male_reading_transformed.set_index("id", inplace=True)

male_reading_transformed.head()

,state,gender,avg_2009_readingScores,avg_2017_readingScores,readingScores_precentChange
id,,,,,
1,AK,Male,206.724229,203.051302,3.672928
3,AL,Male,211.741931,212.821109,-1.079179
5,AR,Male,210.975858,211.742620,-0.766762
7,AZ,Male,206.913680,213.462670,-6.548990
9,CA,Male,207.055218,210.770608,-3.715390


In [22]:
# Female Math Dataframe 
# Create a filtered dataframe from specific columns
female_reading_cols = ["id", "state", "gender", "avg_2009_readingScores", "avg_2017_readingScores", "readingScores_precentChange"]
female_reading_transformed= female_reading_data[female_reading_cols].copy()

# Rename the column headers
female_reading_transformed = female_reading_transformed.rename(columns={"id": "id",
                                                          "state": "state",
                                                            "gender": "gender",
                                                            "avg_2009_readingScores": "avg_2009_readingScores",
                                                            "avg_2017_readingScores": "avg_2017_readingScores",
                                                          "readingScores_precentChange": "readingScores_precentChange"})

# Clean the data by dropping duplicates and setting the index
female_reading_transformed.set_index("id", inplace=True)

female_reading_transformed.head()

,state,gender,avg_2009_readingScores,avg_2017_readingScores,readingScores_precentChange
id,,,,,
2,AK,Female,215.711234,211.343343,4.367891
4,AL,Female,221.100699,220.006600,1.094099
6,AR,Female,221.564513,220.633010,0.931503
8,AZ,Female,213.189225,217.520227,-4.331002
10,CA,Female,212.637646,220.293138,-7.655493


### Connect Data to Database

In [26]:
conn = sq.connect('db.sqlite')

In [27]:
# Confirm tables
# engine.table_names()

In [30]:
#Male Math
male_math_transformed.to_sql(name='male_math', con=conn, if_exists='replace', index=True)

In [31]:
#Femal Math
female_math_transformed.to_sql(name='female_math', con=conn, if_exists='replace', index=True)

In [32]:
#Male Reading 
male_reading_transformed.to_sql(name='male_reading', con=conn, if_exists='replace', index=True)

In [33]:
#Femal Reading 
female_reading_transformed.to_sql(name='female_reading', con=conn, if_exists='replace', index=True)

In [37]:
pd.read_sql_query('select gender from female_math', con=conn).head()

,gender
0,Female
1,Female
2,Female
3,Female
4,Female
